# Chunk discontinuities in quantized block-based (non-overlapped) DWT-transformed audio signals

Let's research how quantization affects to the reconstruction of the chunks when they are DWT-tansformed.

In [ ]:
import sounddevice as sd
!pip install pywavelets
import pywt
import math
!pip install numpy
import numpy as np
%matplotlib inline
!pip install matplotlib
import matplotlib
import matplotlib.pyplot as plt
!pip install scipy
from scipy import signal
!pip install ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
!pip install pylab
import pylab

## Capture an audio sequence

In [ ]:
def plot(y, xlabel='', ylabel='', title='', marker='.'):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.grid()
    ax.xaxis.set_label_text(xlabel)
    ax.yaxis.set_label_text(ylabel)
    x = np.linspace(0, len(y)-1, num=len(y))
    ax.plot(x, y, marker, markersize=1)
    plt.show(block=False)

In [ ]:
fs = 44100
duration = 80000/44100  # seconds
signal = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype=np.int16)
print("Say something!")
while sd.wait():
    pass
print("done")
signal = signal.flatten()

In [ ]:
plot(signal, "sample", "amplitude", "original")

### Configuration of the experiment

In [ ]:
levels = 3           # Number of levels of the DWT
filters_name = "db5"
#filters_name = "haar"
#filters_name = "db11"
#filters_name = "db20"
#filters_name = "bior2.2"
#filters_name = "bior3.5"
#filters_name = "rbio2.2"
wavelet = pywt.Wavelet(filters_name)
signal_mode_extension = "per"
quantization_step = 256
chunk_size = 128
chunk_number = 150

## Dead-zone quantizer

In [ ]:
def deadzone_quantizer(x, quantization_step):
    k = (x / quantization_step).astype(np.int)
    return k

def deadzone_dequantizer(k, quantization_step):
    y = quantization_step * k
    return y

## Extract 3 consecutive chunks from the audio sequence

In [ ]:
left_chunk = signal[chunk_size * (chunk_number - 1) : chunk_size * chunk_number]
center_chunk = signal[chunk_size * chunk_number : chunk_size * (chunk_number + 1)]
right_chunk = signal[chunk_size * (chunk_number + 1) : chunk_size * (chunk_number + 2)]
chunks = np.concatenate([left_chunk, center_chunk, right_chunk])
pylab.plot(np.linspace(0, len(left_chunk)-1, len(left_chunk)), left_chunk)
pylab.plot(np.linspace(len(left_chunk), len(left_chunk) + len(center_chunk) - 1, len(center_chunk)), center_chunk)
pylab.plot(np.linspace(len(left_chunk) + len(center_chunk), len(left_chunk) + len(center_chunk) + len(right_chunk) - 1, len(right_chunk)), right_chunk)
pylab.xlabel("sample")
pylab.ylabel("amplitude")
pylab.title("3 chunks")
pylab.show()

## Algorithm 0: Quantization of the chunks in the DWT domain

* For each chunk:
    * Transform.
    * Quantize.
    * Dequantize.
    * Detransform.

In [ ]:
def transform_and_quantize(chunk):
    decomposition = pywt.wavedec(chunk, wavelet=wavelet, level=levels, mode=signal_mode_extension)
    quantized_decomposition = []
    for subband in decomposition:
        quantized_subband = deadzone_quantizer(subband, quantization_step)
        quantized_decomposition.append(quantized_subband)
    return quantized_decomposition
    
def dequantize_and_detransform(decomposition):
    dequantized_decomposition = []
    for subband in decomposition:
        dequantized_subband = deadzone_dequantizer(subband, quantization_step)
        dequantized_decomposition.append(dequantized_subband)
    chunk = pywt.waverec(dequantized_decomposition, wavelet=wavelet, mode=signal_mode_extension)
    return chunk

def reconstruct_chunk(chunk):
    quantization_indexes = transform_and_quantize(chunk)
    reconstructed_chunk = dequantize_and_detransform(quantization_indexes)
    return reconstructed_chunk
    
reconstructed_left_chunk = reconstruct_chunk(left_chunk)
reconstructed_center_chunk = reconstruct_chunk(center_chunk)
reconstructed_right_chunk = reconstruct_chunk(right_chunk)
ideal_chunks_reconstruction = reconstruct_chunk(chunks)

pylab.plot(np.linspace(0, len(reconstructed_left_chunk)-1, len(reconstructed_left_chunk)), reconstructed_left_chunk)
pylab.plot(np.linspace(len(reconstructed_left_chunk), len(reconstructed_left_chunk) + len(reconstructed_center_chunk) - 1, len(reconstructed_center_chunk)), reconstructed_center_chunk)
pylab.plot(np.linspace(len(reconstructed_left_chunk) + len(reconstructed_center_chunk), len(reconstructed_left_chunk) + len(reconstructed_center_chunk) + len(reconstructed_right_chunk) - 1, len(reconstructed_right_chunk)), reconstructed_right_chunk)
pylab.plot(np.linspace(0, len(ideal_chunks_reconstruction)-1, len(ideal_chunks_reconstruction)), ideal_chunks_reconstruction, linestyle='dotted', label="ideal")
pylab.xlabel("sample")
pylab.ylabel("amplitude")
pylab.title("reconstruction of the 3 chunks")
pylab.legend(loc='upper right')
pylab.show()